## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
#Вызовем функци import
import pandas as pd

In [2]:
#Указываем путь и сохраняем в переменной
data = pd.read_csv('/datasets/data.csv')

In [3]:
#Напечатаем первые 10 строк таблицы для наглядности
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
#Посмотрим общие данные о таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

# При изучении общей информации таблицы были выявлены следущие замечания:
1. В данных существуют пропуски
2. В столбцах таблицы указан неверный формат данных - это можеть повлечь за собой не правильные выводы.
3. Обнаружены дубликаты 
4. Обнаруженны не корректные данные по столбцам 'childre', 'days_employed', 'dob_years'
Приступим к исправлению замечаний.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
#Узнаем сумму пропущенных значений таблицы
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
#Получили пропуски только в двух столбцах: days_employed и total_income.
#Проверим в каких профессиях отсутствуют данные по зарплате. 
#Столбец "days_employed" не нужен нам для анализа, не будем тратить на него время

data.loc[data['total_income'].isnull(), 'income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [8]:
#Посчитаем в процентном соотнощении пропуски по зарплате в cпециальностях
data.loc[data['total_income'].isnull(), 'income_type'].value_counts(normalize=True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

#Процентное соотнощение пропусков по зарплате в cпециальностях:
сотрудник          50.8 %
компаньон          23.3 %
пенсионер          18.9 %
госслужащий        6.7 %
предприниматель    0.04 %

In [9]:
#Напишем функцию замены пропусков в столбце запралты на средние значения
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))

In [10]:
#Убедимся, что пропущенных значений больше нет
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

 В данных обнаружены пропуски в стобцах 'days_employed' (обший трудовой стаж) и 'total_income' ( ежемесячный доход). Восполнили отсутствующие значения в столбце 'total_income'. Отсутствие данных, вероятней всего, обусловленно отказом в предоставлении этих данных клиентом, или же технической ошибкой. Повторная проверка данных показала, что таблица не имеет пропусков.

### Замена типа данных

In [12]:
#Посмотрим общие данные таблицы data еще раз
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [13]:
#Изменим тип данных столбца 'total_income' с float64 на подходящий нам int64
data['total_income'] = data['total_income'].astype('int')

In [14]:
#Тоже самое повторим с столбцом 'days_employed'
#data['days_employed'] = data['days_employed'].astype('int')

In [15]:
#Выполним проверку на изменение данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

При изучении общей информации было выявлено замечание о не верном типе данных 
(days_employed и total_income имеют формат 'float'),
что могло бы дать не верные выводы в ходе анализа.
Перевод в нужный тип данных произвден успешно, результаты проверки это подтверждают.

### Обработка дубликатов

In [16]:
#При визуальном изучении таблицы был выявлен разный регистр в столбце 'education'
#Приведем регистр с единому стандарту
data['education']=data['education'].str.lower()

In [17]:
#Теперь узнаем количество дубликатов в таблице
data.duplicated().sum()

71

In [18]:
#Удалим все дубликаты c сохранением индекса
data = data.drop_duplicates().reset_index(drop=True)

In [19]:
#Выполним проверку
data.duplicated().sum()

0

### Вывод

Был произведен поиск дубликатов, а затем очистка от них в таблице data. Причинами появления дупликатов мог стать как человечкий фактор, так и ошибка в программе. Для поиска дублткатов был использован метод .diplicated().

### Лемматизация

In [20]:
# Запустим цикл лемматизации
from pymystem3 import Mystem
m = Mystem()
def lemmatize(text):
    lemma = m.lemmatize(text)
    return lemma

In [21]:
#распечатаем новую таблицу
data['purpose'].apply(lemmatize)

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21449                  [операция,  , с,  , жилье, \n]
21450               [сделка,  , с,  , автомобиль, \n]
21451                              [недвижимость, \n]
21452    [на,  , покупка,  , свой,  , автомобиль, \n]
21453             [на,  , покупка,  , автомобиль, \n]
Name: purpose, Length: 21454, dtype: object

### Вывод

Была произведенна леммитизация данных столбцу "purpose" для выявления целей кредитования клиентов. Также был создан новый столбец "purpose_group" для определения каждого id клиента к определенной группе по цели кредитования.

### Вывод

### Категоризация данных

In [22]:
#Найдем уникальные цели по кредиту в столбце 'purpose'
unique = data['purpose'].value_counts()
unique

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [23]:
from pymystem3 import Mystem
m = Mystem()
def lemmatize(text):
    lemma = m.lemmatize(text)
    return lemma
from collections import Counter

In [24]:
#Применим лематизацию к столбцу еще раз
data['purpose'].apply(lemmatize).value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

In [25]:
#Выделим ключевые слова из списка выше
tcel = [ "свадьба", "недвижимость", "жилье", "автомобиль", "образование"]

In [26]:
#Заменим полученный список лемм ключевое слово из списка "tcel"
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in tcel:
        if word in lemma:
            lemma = word
    return lemma    

In [27]:
#Добавим новый столбец в таблицу data - data['purpose_group'].
#Данный столбец будет относить каждый id к определенной группе заемщиков по цели кредита
data['purpose_group'] = data['purpose'].apply(lemmatize)        

In [28]:
#Распечатаем новую таблицу
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [29]:
#Посмотрим популярность целей по кредиту
data['purpose_group'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_group, dtype: int64

In [30]:
#Объединим цели по кредиту "недвижимость" и "жилье", так как они идентичны
data.loc[data['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'

In [31]:
#Проверим результат
data['purpose_group'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

### Вывод

Разобрали id клиентов по кредитным целям - теперь их всего 4) Из таблицы можно сделать вывод, что самая полулярная цель кредита - недвижимость, а категория 60+ считает, что жениться никогда не поздно. Меньше всего кредитами интересуются предприниматели, безработные, находящиеся в декрете и студенты.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
#Посмотрим еще раз общую информацию о столбце 'children'
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21454.000000,19351.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.539946,63046.497661,43.271231,0.817097,0.973898,0.081150,1.674316e+05
std,1.383444,140827.311974,12.570822,0.548674,1.421567,0.273072,9.806060e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.076230e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.518870e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.024170e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [33]:
#Из общей информации по таблице, мы видим не достоверные данные, поэтому напишем фильтр для столбца "children"
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1

In [34]:
def children_group(children):
    if children == 0: return 'Без детей'
    if children == 1: return 'В семье один ребенок'
    if children == 2: return 'В семье 2 детей'
    if children == 3: return 'В семье 3 детей'
    if children == 4: return 'В семье 4 детей'
    if children == 5: return 'В семье 5 детей'
    

In [35]:
#Теперь применим функции к столбцу и сохраним ее в новом столбце
data['child_1'] = data['children'].apply(children_group)

In [36]:
#Соберем сводную таблицу 
data_pivot = data.pivot_table(index = ['child_1'], columns = 'debt', values = 'gender', aggfunc = 'count')

In [37]:
#Произведем расчет по не выплатам кредита и сохраним в новом столбце
data_pivot['debt_index'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])


In [38]:
#Распечатаем таблицу и посмотрим результат
data_pivot

debt,0,1,debt_index
child_1,,,
Без детей,13028.0,1063.0,0.075438
В семье 2 детей,1926.0,202.0,0.094925
В семье 3 детей,303.0,27.0,0.081818
В семье 4 детей,37.0,4.0,0.097561
В семье 5 детей,9.0,NaN,NaN
В семье один ребенок,4410.0,445.0,0.091658


### Вывод

Клиенты, которые не имеют детей чаще исполняют свои обязательства по крудиту, чем клиенты, которые имеют 1 или несколько детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
#Соберем сводную таблицу 
data_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931


In [40]:
#Произведем расчет по не выплатам кредита и сохраним в новом столбце
data_pivot['debt_index'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])

In [41]:
data_pivot

debt,0,1,debt_index
family_status,,,
Не женат / не замужем,2536,274,0.097509
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452


### Вывод

Согласно таблице, к не исполнениею обязательств по кредиту склоны клиенты, которые не находятся в браке. Более ответственными оказались клиенты состоящие в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
#Посмотри на столбец total_income
data['total_income'].describe()

count    2.145400e+04
mean     1.674316e+05
std      9.806060e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.518870e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [43]:
#Разобъем всех id по возрасту на 4 группы : 'до 30', '30-45', '45-60', '60+' 
def income_group(total_income):
    if total_income <= 50000: return 'до 50000'
    if 50000 < total_income <= 100000: return '50000-100000'
    if 100000 < total_income <= 200000: return '100000-200000'
    else: 
        return '200000+'
    
data['income_group'] = data['total_income'].apply(income_group)
print(data['income_group'].value_counts())

100000-200000    11423
200000+           5568
50000-100000      4091
до 50000           372
Name: income_group, dtype: int64


In [44]:
#Соберем сводную таблицу 
data_pivot = data.pivot_table(index = ['income_group'], columns = 'debt', values = 'gender', aggfunc = 'count')

In [45]:
##Произведем расчет по не выплатам кредита и сохраним в новом столбце
data_pivot['debt_index'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])

In [46]:
#Распечатаем таблицу и посмотрим результат
data_pivot

debt,0,1,debt_index
income_group,,,
100000-200000,10424,999,0.087455
200000+,5180,388,0.069684
50000-100000,3760,331,0.080909
до 50000,349,23,0.061828


### Вывод

Согласно таблице, к неисполнению обязательств по кредиту склоны клиенты с доходом от 100000 до 200000 р. Более ответственными оказались клиенты С доходом до 50000 р.

- Как разные цели кредита влияют на его возврат в срок?

In [47]:
#Соберем сводную таблицу 
data_pivot = data.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'gender', aggfunc = 'count')


In [48]:
#Произведем расчет по не выплатам кредита и сохраним в новом столбце
data_pivot['debt_index'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])


In [49]:
#Распечатаем таблицу и посмотрим результат
data_pivot

debt,0,1,debt_index
purpose_group,,,
автомобиль,3903,403,0.093590
недвижимость,10029,782,0.072334
образование,3643,370,0.092200
свадьба,2138,186,0.080034


### Вывод

Клиенты, которые берут кредит на покупку автомобиля чаще всего не испролняют обязательства по кредиту: процент не возврата: 9.3%, а вот с недвижимостью рисков меньше всего: 7.3%

### Шаг 4. Общий вывод

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
Согласно таблице, клиенты, которые не имеют детей реже имеют не возврат кредита.
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
Согласно таблице, к не возврату кредита склоны клиенты с статусом "Не женат / не замужем", более ответственными оказались клиенты с статусом "женат / замужем"
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Согласно таблице, к не возврату кредита склоны клиенты с категории "75000-100000", более ответственными оказались клиенты в категории "до 50000"
- Как разные цели кредита влияют на его возврат в срок?
Опаснее всего давать кредит на автомобиль - процент не возврата: 9.3%, а вот с недвижимостью рисков меньше: 7.3%
